## Extrae datos de idealista mediante técnica de web scrapping

### Dependencias e importaciones

In [1]:
# pip install undetected-chromedriver

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import random
import time
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import undetected_chromedriver as uc

In [5]:
URL_INICIAL_LISTADO = "https://www.idealista.com/venta-viviendas/oropesa-del-mar-castellon/con-sin-inquilinos/"
URL_INMUEBLE = "https://www.idealista.com/inmueble/%INMUEBLE_ID%/"

In [7]:
browser = uc.Chrome()

### 1º hacemos una lectura de las distintas páginas para ir anotando los ids

In [9]:
paginacion = 1
ids = []
resultados = []
# while True:
while paginacion == 1:
    print('Página: ', paginacion)
    url = f'{URL_INICIAL_LISTADO}{"pagina-" + str(paginacion) + ".htm" if paginacion > 1 else ""}'
    browser.get(url)
    time.sleep(random.randint(8, 12))

    # Trata de marcar el sí en la información de cookies
    try:
        browser.find_element("xpath", '//*[@id="didomi-notice-agree-button"]').click()
    except:
        pass

    html = browser.page_source
    soup = bs(html, 'html.parser')

    pagina_actual = int(soup.find('main', {'class': 'listing-items'}).find('div', {'class': 'pagination'}).find('li', {'class': 'selected'}).text)
    if paginacion == pagina_actual:
        # articles = soup.find('main', {'class': 'listing-items'}).find_all('article')
        articles = soup.find('main', {'class': 'listing-items'}).find_all('article', attrs={'data-element-id': True})
    else:
        break

    paginacion += 1


for article in articles:
    # ID del artículo
    element_id = article.get('data-element-id')

    # Enlace y título
    link_tag = article.find('a', class_='item-link')
    enlace = link_tag.get('href') if link_tag else ''
    titulo = link_tag.get('title') if link_tag else ''

    # Precio
    price_tag = article.find('span', class_='item-price')
    precio = price_tag.get_text(strip=True) if price_tag else ''

    # Detalles (habitaciones, m2, planta...)
    detalles_tags = article.find_all('span', class_='item-detail')
    detalles = ' | '.join([d.get_text(strip=True) for d in detalles_tags])

    # Descripción
    desc_tag = article.find('div', class_='item-description')
    descripcion = desc_tag.get_text(strip=True) if desc_tag else ''

    # Etiquetas (ej: Vistas al mar)
    tag_container = article.find('div', class_='listing-tags-container')
    etiquetas = ''
    if tag_container:
        etiquetas = ' | '.join([
            tag.get_text(strip=True) for tag in tag_container.find_all('span', class_='listing-tags')
        ])

    # Diccionario final para este artículo
    datos = {
        'id': element_id,
        'enlace': enlace,
        'titulo': titulo,
        'precio': precio,
        'detalles': detalles,
        'descripcion': descripcion,
        'etiquetas': etiquetas
    }

    resultados.append(datos)
print(resultados)




    # for article in articles:
    #     id_muebles = article.get('data-element-id')
    #     ids.append(id_muebles)
    #     print(id_muebles)
    #     time.sleep(random.randint(1, 3))
    # ids = [muebles for muebles in ids if muebles is not None]

Página:  1
[{'id': '100000581', 'enlace': '/inmueble/100000581/', 'titulo': "Piso en termalismo, Torre Bellver - La Renegá - Les Platgete'S, Oropesa del Mar", 'precio': '590.000€', 'detalles': '4 hab. | 161 m² | Planta 3ª exterior con ascensor', 'descripcion': 'Disfrutar, unos días, unas vacaciones, una temporada larga, durante todo el año, sea cual sea su elección, usted dispone de todos los servicios y más en Benicassim.\nBenicàssim cuenta con un clima y una orografía excelentes que le permiten ofrecer una amplia oferta de actividades deportivas durante todo el año. Tanto si', 'etiquetas': 'Vistas al mar'}, {'id': '105575288', 'enlace': '/inmueble/105575288/', 'titulo': "Dúplex en calle Forcall, 8, Torre Bellver - La Renegá - Les Platgete'S, Oropesa del Mar", 'precio': '550.000€', 'detalles': '4 hab. | 165 m² | Planta 2ª exterior con ascensor', 'descripcion': 'Espectacular ático duplex con vistas al Mar a 5 minutos de BenicassimDisfrutar, unos días, unas vacaciones, una temporada lar

In [ ]:
df_resultados = pd.DataFrame(resultados)
df_resultados
